# Surface contextualization

> Module for surface contextualization of cortical indices

In [ ]:
#| default_exp surface_contextualization

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def plot_surface(
    surface:str, # Surface to plot on (default: fslr32k). Valid choices are “fslr32k”, “fsaverage”, “fsaverage3”, “fsaverage4”, “fsaverage5”, “fsaverage6”, “civet41k”, “civet164k”.
    values, # numpy array of values to plot (has to match the number of vertices in the surface)
    label_text:str, # text to label the plot
    color_range:tuple=None, # color range to use
    cmap="Blues" # color map to use
    )->object: # Plot 
    "Plots metric values on surface"
    
    # Plot cortical surfaces with values as the data, label_text as
    # the labels, and color_range as the limits of the color bar.
    from brainstat.datasets import fetch_mask, fetch_template_surface

    # Load behavioral markers
    pial_left, pial_right = fetch_template_surface(surface, join=False)
    pial_combined = fetch_template_surface(surface, join=True)
    mask = fetch_mask(surface)

    
    from brainspace.plotting import plot_hemispheres
    import numpy as np

    if not color_range:
        color_range = (np.nanmin(values), np.nanmax(values))

    return plot_hemispheres(
        pial_left,
        pial_right,
        values,
        color_bar=True,
        color_range=color_range,
        label_text=label_text,
        cmap=cmap,
        embed_nb=True,
        size=(1400, 200),
        zoom=1.45,
        nan_color=(0.5, 0.5, 0.5, 1),
        cb__labelTextProperty={"fontSize": 12},
        interactive=False,
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()